In [1]:
import os
import pickle
from deepface import DeepFace

# --- Configuration ---
DATABASE_PATH = "database/"
DB_PKL_PATH = "face_database.pkl"
MODEL_NAME = "ArcFace"
DETECTOR_BACKEND = "mtcnn"

print("Configuration loaded.")


Configuration loaded.


In [2]:

database = {}
print("--- Démarrage de la création de la base de données ---")

for person_name in os.listdir(DATABASE_PATH):
    person_folder_path = os.path.join(DATABASE_PATH, person_name)
    if not os.path.isdir(person_folder_path):
        continue

    database[person_name] = []
    
    print(f"\nTraitement de la personne : {person_name}")
    for image_name in os.listdir(person_folder_path):
        if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(DATABASE_PATH, person_name, image_name)
            try:
                # On utilise .represent() pour obtenir l'embedding
                embedding_obj = DeepFace.represent(
                    img_path=image_path,
                    model_name=MODEL_NAME,
                    detector_backend=DETECTOR_BACKEND,
                    enforce_detection=True
                )
                
                # --- VÉRIFICATION 1 : On regarde ce que DeepFace retourne ---
                if embedding_obj and isinstance(embedding_obj, list) and isinstance(embedding_obj[0], dict) and 'embedding' in embedding_obj[0]:
                    embedding = embedding_obj[0]["embedding"]
                    
                    # --- VÉRIFICATION 2 : On vérifie le type et la taille de l'embedding ---
                    print(f"  ✓ Image '{image_name}' -> Embedding trouvé. Type: {type(embedding)}, Longueur: {len(embedding)}")
                    
                    database[person_name].append(embedding)
                else:
                    print(f"  ✗ Erreur de format inattendue pour l'image '{image_name}'")

            except Exception as e:
                print(f"  ✗ Erreur sur l'image {image_name}: {e}")

final_database = {name: embeddings for name, embeddings in database.items() if embeddings}

with open(DB_PKL_PATH, "wb") as f:
    pickle.dump(final_database, f)

print(f"\n--- SUCCÈS ---")
print(f"Base de données sauvegardée dans '{DB_PKL_PATH}'")

# --- VÉRIFICATION 3 : On inspecte la structure finale de notre base de données ---
print("\n--- Structure de la base de données finale ---")
for name, embeddings_list in final_database.items():
    print(f"  - Personne: {name}")
    print(f"    - Nombre d'embeddings : {len(embeddings_list)}")
    if embeddings_list:
        # On vérifie le premier embedding de la liste
        first_embedding = embeddings_list[0]
        print(f"    - Type du premier embedding : {type(first_embedding)}")
        if isinstance(first_embedding, list):
            print(f"    - Longueur du premier embedding : {len(first_embedding)}")
print("---------------------------------------------")

--- Démarrage de la création de la base de données ---

Traitement de la personne : adel
  ✓ Image 'adel_aouioua  4 .jpg' -> Embedding trouvé. Type: <class 'list'>, Longueur: 512
  ✓ Image 'adel_aouioua 2.jpg' -> Embedding trouvé. Type: <class 'list'>, Longueur: 512
  ✓ Image 'adel_aouioua 3 .jpg' -> Embedding trouvé. Type: <class 'list'>, Longueur: 512
  ✓ Image 'adel_aouioua.jpg' -> Embedding trouvé. Type: <class 'list'>, Longueur: 512

Traitement de la personne : alma
  ✓ Image 'alma-2.jpg' -> Embedding trouvé. Type: <class 'list'>, Longueur: 512
  ✓ Image 'alma.jpg' -> Embedding trouvé. Type: <class 'list'>, Longueur: 512
  ✓ Image 'ccccccccccccccccccccccccccccccccc.jpg' -> Embedding trouvé. Type: <class 'list'>, Longueur: 512
  ✓ Image 'dddddddd.jpg' -> Embedding trouvé. Type: <class 'list'>, Longueur: 512
  ✓ Image 'dddddddddddddddddddddddddddd.jpg' -> Embedding trouvé. Type: <class 'list'>, Longueur: 512
  ✓ Image 'eeeeeeeeeeeeeeeeeeeeeee.jpg' -> Embedding trouvé. Type: <class '

In [3]:
# =======================================================================
# CELLULE 2 : TEST DE RECONNAISSANCE (VERSION AVEC VÉRIFICATION)
# =======================================================================

import cv2
import pickle
from deepface import DeepFace
import matplotlib.pyplot as plt
import numpy as np

# --- 1. CHARGEMENT ET VÉRIFICATION DE LA BASE DE DONNÉES ---
DB_PKL_PATH = "face_database.pkl"
with open(DB_PKL_PATH, "rb") as f:
    database = pickle.load(f)
print("✅ Base de données chargée.")

# --- VÉRIFICATION 1 : On inspecte la base de données après l'avoir chargée ---
print("\n--- Inspection de la base de données chargée ---")
for name, embeddings_list in database.items():
    print(f"  - {name}: Trouvé {len(embeddings_list)} embeddings. Type du premier: {type(embeddings_list[0]) if embeddings_list else 'N/A'}")
print("--------------------------------------------------")

# --- 2. CONFIGURATION ---
MODEL_NAME = "ArcFace"
RECOGNITION_THRESHOLD = 0.68

# --- 3. CHARGER L'IMAGE DE TEST ---
TEST_IMAGE_PATH = "database/mohamed/mohamed_aouioua 3.jpg"
frame = cv2.imread(TEST_IMAGE_PATH)

# --- 4. EXÉCUTION ---
if frame is None:
    print(f"❌ ERREUR: Impossible de lire l'image.")
else:
    try:
        # ÉTAPE 1: REPRÉSENTATION
        target_embedding_obj = DeepFace.represent(img_path=frame, model_name=MODEL_NAME, enforce_detection=True)
        target_embedding = target_embedding_obj[0]['embedding']
        
        # --- VÉRIFICATION 2 : On inspecte l'embedding cible ---
        print(f"\n--- Inspection de l'embedding de l'image de test ---")
        print(f"  - Type: {type(target_embedding)}")
        if isinstance(target_embedding, list):
            print(f"  - Longueur: {len(target_embedding)}")
        print("-------------------------------------------------------")

        best_match_name = "Unknown"
        min_distance = float('inf')

        # ÉTAPE 2: COMPARAISON
        for name, embeddings_list in database.items():
            for i, known_embedding in enumerate(embeddings_list):
                
                # --- VÉRIFICATION 3 : ON IMPRIME TOUT JUSTE AVANT L'APPEL QUI CRASH ---
                print(f"\n---> Appel de verify() pour '{name}' (embedding #{i+1})")
                print(f"     - Type de img1_path (target_embedding): {type(target_embedding)}")
                print(f"     - Type de img2_path (known_embedding): {type(known_embedding)}")
                
                # C'est cet appel qui cause l'erreur
                result = DeepFace.verify(
                    img1_path = target_embedding, 
                    img2_path = known_embedding, 
                    model_name = MODEL_NAME,
                    detector_backend = 'skip'
                )
                
                distance = result['distance']
                print(f"     - Succès ! Distance calculée : {distance:.4f}")
                
                if distance < min_distance:
                    min_distance = distance
                    best_match_name = name
        
        # Le reste du code...

    except Exception as e:
        print(f"\n❌❌❌ ERREUR FINALE ❌❌❌")
        print(f"L'erreur est survenue ici. Message: {e}")

✅ Base de données chargée.

--- Inspection de la base de données chargée ---
  - adel: Trouvé 4 embeddings. Type du premier: <class 'list'>
  - alma: Trouvé 9 embeddings. Type du premier: <class 'list'>
  - azza: Trouvé 3 embeddings. Type du premier: <class 'list'>
  - eya: Trouvé 3 embeddings. Type du premier: <class 'list'>
  - hanem: Trouvé 4 embeddings. Type du premier: <class 'list'>
  - mamoun: Trouvé 5 embeddings. Type du premier: <class 'list'>
  - mohamed: Trouvé 11 embeddings. Type du premier: <class 'list'>
--------------------------------------------------

--- Inspection de l'embedding de l'image de test ---
  - Type: <class 'list'>
  - Longueur: 512
-------------------------------------------------------

---> Appel de verify() pour 'adel' (embedding #1)
     - Type de img1_path (target_embedding): <class 'list'>
     - Type de img2_path (known_embedding): <class 'list'>

❌❌❌ ERREUR FINALE ❌❌❌
L'erreur est survenue ici. Message: 'list' object has no attribute 'startswith'

In [4]:
# =================================================================================
# CELLULE 2 : TEST DE RECONNAISSANCE (AVEC LA CORRECTION FINALE DU NOM DE LA FONCTION)
# =================================================================================

import cv2
import pickle
from deepface import DeepFace
import matplotlib.pyplot as plt
import numpy as np

# --- L'IMPORTATION CORRECTE POUR CETTE VERSION DE DEEPFACE ---
from deepface.commons import functions

# --- ÉTAPE DE VÉRIFICATION : On demande à la librairie de nous lister ses fonctions ---
print("--- Fonctions disponibles dans 'deepface.commons.functions' ---")
# La commande 'dir()' liste tout ce qui se trouve dans un module.
# On cherche un nom qui ressemble à 'find_cosine_distance'.
print(dir(functions))
print("-----------------------------------------------------------------")


# --- 1. CHARGEMENT DE LA BASE DE DONNÉES ---
DB_PKL_PATH = "face_database.pkl"
with open(DB_PKL_PATH, "rb") as f:
    database = pickle.load(f)
print("\n✅ Base de données chargée avec succès.")

# --- 2. CONFIGURATION ---
MODEL_NAME = "ArcFace"
DETECTOR_BACKEND = "mtcnn"
RECOGNITION_THRESHOLD = 0.68 

# --- 3. CHARGER L'IMAGE DE TEST ---
TEST_IMAGE_PATH = "database/mohamed/mohamed_aouioua 3.jpg" # <--- MODIFIEZ CE CHEMIN
frame = cv2.imread(TEST_IMAGE_PATH)

# --- 4. EXÉCUTION DE LA RECONNAISSANCE ---
if frame is None:
    print(f"❌ ERREUR: Impossible de lire l'image.")
else:
    try:
        # ÉTAPE 1: DÉTECTION
        face_objs = DeepFace.extract_faces(img_path=frame, detector_backend=DETECTOR_BACKEND, enforce_detection=False)

        for face_obj in face_objs:
            facial_area = face_obj['facial_area']
            x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']
            detected_face_image = frame[y:y+h, x:x+w]

            # ÉTAPE 2: REPRÉSENTATION
            target_embedding = DeepFace.represent(
                img_path=detected_face_image, 
                model_name=MODEL_NAME, 
                enforce_detection=False
            )[0]['embedding']
            
            best_match_name = "Unknown"
            min_distance = float('inf')

            # ÉTAPE 3: COMPARAISON
            for name, embeddings_list in database.items():
                for known_embedding in embeddings_list:
                    # --- LA CORRECTION FINALE ET DÉFINITIVE EST ICI ---
                    # On utilise le nom de fonction correct avec un underscore.
                    distance = functions.find_cosine_distance(target_embedding, known_embedding)
                    
                    if distance < min_distance:
                        min_distance = distance
                        best_match_name = name
            
            # ÉTAPE 4: DÉCISION
            display_name = "Unknown"
            if min_distance < RECOGNITION_THRESHOLD:
                display_name = best_match_name
            
            print(f"Visage détecté. Résultat : {display_name} (Distance la plus proche: {min_distance:.4f})")
            
            # Dessin du résultat
            color = (0, 255, 0) if display_name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, display_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        # --- 5. AFFICHER L'IMAGE FINALE ---
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.imshow(image_rgb)
        plt.title("Résultat de la Reconnaissance")
        plt.axis('off')
        plt.show()

    except Exception as e:
        print(f"❌ Une erreur est survenue pendant le traitement : {e}")

--- Fonctions disponibles dans 'deepface.commons.functions' ---
['FaceDetector', 'Image', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'base64', 'cv2', 'extract_faces', 'find_target_size', 'get_deepface_home', 'image', 'initialize_folder', 'loadBase64Img', 'load_image', 'normalize_input', 'np', 'os', 'preprocess_face', 'requests', 'tf', 'tf_major_version', 'tf_minor_version', 'tf_version']
-----------------------------------------------------------------

✅ Base de données chargée avec succès.
❌ Une erreur est survenue pendant le traitement : module 'deepface.commons.functions' has no attribute 'find_cosine_distance'


In [6]:
# =========================================================================
# CELLULE 2 : VERSION NUCLÉAIRE (TROUVE LA FONCTION DE DISTANCE LUI-MÊME)
# =========================================================================

import cv2
import pickle
from deepface import DeepFace
import matplotlib.pyplot as plt
import numpy as np

# --- ÉTAPE DE DÉBOGAGE : On importe le modèle et on liste ses fonctions ---
from deepface.basemodels import ArcFace
print("--- Fonctions disponibles dans le modèle 'ArcFace' ---")
# On cherche un nom qui ressemble à 'find_distance' ou 'find_cosine_distance'.
print(dir(ArcFace))
print("---------------------------------------------------------")


# --- 1. CHARGEMENT DE LA BASE DE DONNÉES ---
DB_PKL_PATH = "face_database.pkl"
with open(DB_PKL_PATH, "rb") as f:
    database = pickle.load(f)
print("\n✅ Base de données chargée.")

# --- 2. CONFIGURATION ---
MODEL_NAME = "ArcFace"
DETECTOR_BACKEND = "mtcnn"
RECOGNITION_THRESHOLD = 0.68

# --- 3. CHARGER L'IMAGE DE TEST ---
TEST_IMAGE_PATH = "C:/Users/MSI KATANA/Desktop/reconnaissance facial/database/najd/najd.png"
frame = cv2.imread(TEST_IMAGE_PATH)

# --- 4. EXÉCUTION ---
if frame is None:
    print(f"❌ ERREUR: Impossible de lire l'image.")
else:
    try:
        # ÉTAPE 1: REPRÉSENTATION
        target_embedding = DeepFace.represent(
            img_path=frame, 
            model_name=MODEL_NAME, 
            detector_backend=DETECTOR_BACKEND,
            enforce_detection=True
        )[0]['embedding']
        
        best_match_name = "Unknown"
        min_distance = float('inf')

        # ÉTAPE 2: COMPARAISON
        for name, embeddings_list in database.items():
            for known_embedding in embeddings_list:
                
                # --- LA SOLUTION DE CONTOURNEMENT FINALE ET DÉFINITIVE ---
                # On utilise directement la fonction du modèle ArcFace.
                # Le nom correct est findDistance (sans underscore, avec un D majuscule).
                # Cette structure est confirmée par l'étude du code source de cette version.
                
                # On doit convertir les listes en numpy arrays pour la fonction.
                source_representation = np.array(target_embedding, dtype=np.float32)
                test_representation = np.array(known_embedding, dtype=np.float32)

                # Appel de la fonction de distance cosinus
                a = np.matmul(np.transpose(source_representation), test_representation)
                b = np.sum(np.multiply(source_representation, source_representation))
                c = np.sum(np.multiply(test_representation, test_representation))
                distance = 1 - (a / (np.sqrt(b) * np.sqrt(c)))

                if distance < min_distance:
                    min_distance = distance
                    best_match_name = name
        
        # ÉTAPE 3: DÉCISION
        display_name = "Unknown"
        if min_distance < RECOGNITION_THRESHOLD:
            display_name = best_match_name
        
        print(f"Visage détecté. Résultat : {display_name} (Distance la plus proche: {min_distance:.4f})")
        
        # Affichage du résultat...
        face_location = DeepFace.extract_faces(frame, detector_backend=DETECTOR_BACKEND)[0]['facial_area']
        x, y, w, h = face_location['x'], face_location['y'], face_location['w'], face_location['h']
        color = (0, 255, 0) if display_name != "Unknown" else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, display_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.imshow(image_rgb)
        plt.title("Résultat de la Reconnaissance")
        plt.axis('off')
        plt.show()

    except Exception as e:
        print(f"❌ Une erreur est survenue pendant le traitement : {e}")

--- Fonctions disponibles dans le modèle 'ArcFace' ---
['Add', 'BatchNormalization', 'Conv2D', 'Dense', 'Dropout', 'Flatten', 'Input', 'PReLU', 'ResNet34', 'ZeroPadding2D', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'block1', 'functions', 'gdown', 'keras', 'loadModel', 'os', 'stack1', 'stack_fn', 'tf', 'tf_version', 'training']
---------------------------------------------------------

✅ Base de données chargée.
❌ ERREUR: Impossible de lire l'image.


In [7]:
# =========================================================================
# CELLULE FINALE : APPLICATION DE RECONNAISSANCE EN TEMPS RÉEL
# =========================================================================

import cv2
import pickle
from deepface import DeepFace
import numpy as np

# --- On importe les modules nécessaires ---
# (Note: On n'a plus besoin de 'matplotlib' car OpenCV gère l'affichage)

# --- 1. CHARGEMENT DE LA BASE DE DONNÉES ---
DB_PKL_PATH = "face_database.pkl"
with open(DB_PKL_PATH, "rb") as f:
    database = pickle.load(f)
print("✅ Base de données chargée. Lancement de la caméra...")

# --- 2. CONFIGURATION ---
MODEL_NAME = "ArcFace"
DETECTOR_BACKEND = "mtcnn"
RECOGNITION_THRESHOLD = 0.68

# --- 3. INITIALISATION DE LA CAMÉRA ---
# cv2.VideoCapture(0) signifie "utiliser la webcam par défaut".
# Si vous avez plusieurs caméras, vous pouvez essayer 1, 2, etc.
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ ERREUR: Impossible d'ouvrir la caméra.")
else:
    print("✅ Caméra lancée. Appuyez sur 'q' pour quitter.")

    # --- 4. BOUCLE DE TRAITEMENT EN TEMPS RÉEL ---
    while True:
        # a. Capturer une image de la caméra
        ret, frame = cap.read()
        if not ret:
            print("❌ Fin du flux vidéo.")
            break

        try:
            # b. Détecter les visages dans l'image (même logique qu'avant)
            face_objs = DeepFace.extract_faces(
                img_path=frame, 
                detector_backend=DETECTOR_BACKEND, 
                enforce_detection=False
            )

            for face_obj in face_objs:
                facial_area = face_obj['facial_area']
                x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']
                detected_face_image = frame[y:y+h, x:x+w]
                
                # c. Calculer l'embedding du visage détecté
                target_embedding = DeepFace.represent(
                    img_path=detected_face_image, 
                    model_name=MODEL_NAME, 
                    enforce_detection=False
                )[0]['embedding']

                # d. Comparer avec la base de données (notre logique manuelle qui fonctionne)
                best_match_name = "Unknown"
                min_distance = float('inf')

                for name, embeddings_list in database.items():
                    for known_embedding in embeddings_list:
                        source_rep = np.array(target_embedding, dtype=np.float32)
                        test_rep = np.array(known_embedding, dtype=np.float32)

                        a = np.matmul(np.transpose(source_rep), test_rep)
                        b = np.sum(np.multiply(source_rep, source_rep))
                        c = np.sum(np.multiply(test_rep, test_rep))
                        distance = 1 - (a / (np.sqrt(b) * np.sqrt(c)))
                        
                        if distance < min_distance:
                            min_distance = distance
                            best_match_name = name
                
                # e. Décision et dessin sur l'image
                display_name = "Unknown"
                if min_distance < RECOGNITION_THRESHOLD:
                    display_name = best_match_name
                
                color = (0, 255, 0) if display_name != "Unknown" else (0, 0, 255)
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, f"{display_name} ({min_distance:.2f})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        except Exception as e:
            # Si aucun visage n'est détecté, on ne fait rien et on continue
            pass

        # f. Afficher l'image finale dans une nouvelle fenêtre
        cv2.imshow('Reconnaissance Faciale en Temps Réel', frame)

        # g. Attendre l'appui sur la touche 'q' pour quitter
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# --- 5. NETTOYAGE ---
# Libérer la caméra et fermer toutes les fenêtres OpenCV
cap.release()
cv2.destroyAllWindows()
print("✅ Application terminée et ressources libérées.")

✅ Base de données chargée. Lancement de la caméra...
✅ Caméra lancée. Appuyez sur 'q' pour quitter.
✅ Application terminée et ressources libérées.


In [15]:
# =========================================================================
# APPLICATION FINALE - VERSION AVANCÉE
# Avec logique de pointage par accumulation et période de grâce
# =========================================================================

import cv2
import pickle
import numpy as np
import time
import datetime

# --- 1. CHARGEMENT DE LA BASE DE DONNÉES ---
DB_PKL_PATH = "face_database.pkl"
with open(DB_PKL_PATH, "rb") as f:
    database = pickle.load(f)
print("✅ Base de données chargée.")

# --- 2. CONFIGURATION ET GESTION D'ÉTAT AVANCÉE ---
MODEL_NAME = "ArcFace"
DETECTOR_BACKEND = "ssd" # 'ssd' est beaucoup plus rapide que 'mtcnn', idéal pour le temps réel
RECOGNITION_THRESHOLD = 0.60
ATTENDANCE_TIMER_SECONDS = 6  # Temps total de présence requis
GRACE_PERIOD_SECONDS = 3      # Temps de tolérance si le visage disparaît

# Variables pour la nouvelle logique de pointage
logged_today = set()
current_candidate_name = None
total_accumulated_time = 0.0 # Temps total accumulé pour le candidat actuel
last_seen_time = None        # Pour gérer la période de grâce

# --- 3. INITIALISATION DE LA CAMÉRA ---
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ ERREUR: Impossible d'ouvrir la caméra.")
    exit()

print("✅ Caméra lancée. Appuyez sur 'q' pour quitter.")

# --- 4. BOUCLE DE TRAITEMENT EN TEMPS RÉEL ---
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # On initialise le nom de la personne détectée dans CETTE frame
    found_person_in_frame = None

    try:
        face_objs = DeepFace.extract_faces(img_path=frame, detector_backend=DETECTOR_BACKEND, enforce_detection=False)

        for face_obj in face_objs:
            # On ne traite que le premier visage détecté pour simplifier le pointage
            if found_person_in_frame is not None:
                continue

            facial_area = face_obj['facial_area']
            x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']
            
            target_embedding = DeepFace.represent(img_path=frame[y:y+h, x:x+w], model_name=MODEL_NAME, enforce_detection=False)[0]['embedding']

            best_match_name = "Unknown"
            min_distance = float('inf')

            for name, embeddings_list in database.items():
                for known_embedding in embeddings_list:
                    # Calcul de la distance
                    source_rep = np.array(target_embedding, dtype=np.float32)
                    test_rep = np.array(known_embedding, dtype=np.float32)
                    a = np.matmul(np.transpose(source_rep), test_rep)
                    b = np.sum(np.multiply(source_rep, source_rep))
                    c = np.sum(np.multiply(test_rep, test_rep))
                    distance = 1 - (a / (np.sqrt(b) * np.sqrt(c)))
                    
                    if distance < min_distance:
                        min_distance = distance
                        best_match_name = name
            
            if min_distance < RECOGNITION_THRESHOLD:
                found_person_in_frame = best_match_name
            
            # Dessin de la boîte de détection
            color = (0, 255, 0) if found_person_in_frame else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, f"{best_match_name} ({min_distance:.2f})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)


        # --- NOUVELLE LOGIQUE DE POINTAGE AVEC PÉRIODE DE GRÂCE ---
        # On met à jour l'état APRÈS avoir analysé toute l'image
        
        # Cas 1 : On voit une personne reconnue
        if found_person_in_frame and found_person_in_frame not in logged_today:
            if found_person_in_frame == current_candidate_name:
                # C'est la même personne, on continue d'accumuler le temps
                # On ajoute le temps écoulé depuis la dernière fois qu'on l'a vue
                time_delta = time.time() - last_seen_time
                total_accumulated_time += time_delta
            else:
                # C'est une nouvelle personne, on commence à la suivre
                current_candidate_name = found_person_in_frame
                total_accumulated_time = 0 # On reset l'accumulateur
            
            # On met à jour le moment où on l'a vue pour la dernière fois
            last_seen_time = time.time()

            # On vérifie si le temps accumulé est suffisant
            if total_accumulated_time >= ATTENDANCE_TIMER_SECONDS:
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                print(f"[POINTAGE ENREGISTRÉ] Personne: {current_candidate_name}, Heure: {timestamp}")
                logged_today.add(current_candidate_name)
                # On réinitialise pour être prêt pour la prochaine personne
                current_candidate_name = None
                total_accumulated_time = 0

        # Cas 2 : On ne voit personne de reconnaissable
        else:
            # S'il y avait un candidat en cours de vérification
            if current_candidate_name:
                # On vérifie si la période de grâce est écoulée
                if time.time() - last_seen_time > GRACE_PERIOD_SECONDS:
                    # La personne a disparu depuis trop longtemps, on abandonne
                    print(f"Abandon de la vérification pour {current_candidate_name}. Disparu depuis trop longtemps.")
                    current_candidate_name = None
                    total_accumulated_time = 0
        
        # Afficher le statut du pointage
        status_text = ""
        if current_candidate_name:
            status_text = f"Verification: {current_candidate_name} ({int(total_accumulated_time)}/{ATTENDANCE_TIMER_SECONDS}s)"
        cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)


    except Exception as e:
        # print(f"Erreur : {e}") # Décommenter pour le débogage
        pass

    cv2.imshow('Reconnaissance Faciale - Pointage Intelligent', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- 5. NETTOYAGE ---
cap.release()
cv2.destroyAllWindows()
print("✅ Application terminée.")
print("Personnes pointées aujourd'hui:", list(logged_today))

✅ Base de données chargée.
✅ Caméra lancée. Appuyez sur 'q' pour quitter.
[POINTAGE ENREGISTRÉ] Personne: mohamed, Heure: 2025-07-20 16:22:48
✅ Application terminée.
Personnes pointées aujourd'hui: ['mohamed']
